In [1]:
#Use this link to understnd API calls and the steps below
#https://docs.dimensions.ai/dsl/getting-started.html

In [18]:
# %pip install tqdm
# %pip install plotly
# %pip install pandas
# %pip install dimcli
# %pip install --upgrade pip
%pip install mysql.connector

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     -- ------------------------------------- 0.7/11.9 MB 13.9 MB/s eta 0:00:01
     ----- ---------------------------------- 1.6/11.9 MB 16.9 MB/s eta 0:00:01
     ------- -------------------------------- 2.2/11.9 MB 15.7 MB/s eta 0:00:01
     --------- ------------------------------ 2.8/11.9 MB 14.7 MB/s eta 0:00:01
     ------------ --------------------------- 3.7/11.9 MB 15.8 MB/s eta 0:00:01
     --------------- ------------------------ 4.5/11.9 MB 16.0 MB/s eta 0:00:01
     ------------------ --------------------- 5.4/11.9 MB 16.3 MB/s eta 0:00:01
     -------------------- ------------------- 6.2/11.9 MB 16.5 MB/s eta 0:00:01
     ----------------------- ---------------- 7.0/11.9 MB 16.6 MB/s eta 0:00:01
     ------------------------- -------------- 7.7/11.9 MB 16.4 MB/s eta 0:00:01
     ----------------------------- ---------- 8.7/11.9 MB 16.3 MB/s eta 0:00:01
     ------------------------------- -------- 9.

In [19]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import mysql.connector
from mysql.connector import Error

In [3]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.8
Method: manual login


In [5]:
names = [
    "Michael Agus",
    "Dorothy Bainton",
    "Charles Balch",
    "Jeffrey Balser",
    "Kevin Catt",
    "Henry Chambers",
    "Richard Clark",
    "Stephanie Eisenbarth",
    "Peng Ji", 
    "Peter Fecci", 
    "Tippi Mackenzie", 
    "Purna Kashyap"
]

In [6]:
def dimensionsPatentSearch(name):
    # Make sure to have dsl initialized somewhere before this function is called
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']

    allIds = [item['id'] for item in listDict]
    allIds_str = ', '.join([f'"{item}"' for item in allIds])
    print(f'Here are all the ids for {name}: ')
    print('[' + allIds_str + ']')

    res2 = dsl.query_iterative(f"""search researchers where id in [{allIds_str}] return researchers[id+obsolete]""")
    listDict2 = res2.json['researchers']
    
    allWorkingIDs = [person['id'] for person in listDict2 if person['obsolete'] == 0]
    print(f'Current Working Id(s): ')
    print(', '.join(allWorkingIDs))
    print(f'++++++++There are {len(allWorkingIDs)} Working IDs+++++++++++')  
    
    finalSet = set()
    count = 0

    if len(allWorkingIDs) == 1:
        res3 = dsl.query_iterative(f"""search patents where researchers = "{allWorkingIDs[0]}" return patents""")
        listDict3 = res3.json['patents']
        
        lowerSet = set()
        for l in listDict3:
            if l['title'].lower() not in lowerSet:
                finalSet.add(l['title'])
                lowerSet.add(l['title'].lower())
            else:
                count += 1
    else:
        for working_id in allWorkingIDs:
            res3 = dsl.query_iterative(f"""search patents where researchers = "{working_id}" return patents""")
            listDict3 = res3.json['patents']
            
            lowerSet = set()
            for l in listDict3:
                if l['title'].lower() not in lowerSet:
                    finalSet.add(l['title'])
                    lowerSet.add(l['title'].lower())
                else:
                    count += 1
    
    print(f'There are {count} number of duplicated patents removed')
    print(f'Final set has {len(finalSet)} number of patents')
    print()
    return finalSet

# Calling the function
dimensionsPatentSearch("Charles Balch")


Starting iteration with limit=1000 skip=0 ...
0-58 / 58 (6.22s)
===
Records extracted: 58
Starting iteration with limit=1000 skip=0 ...


Here are all the ids for Charles Balch: 
["ur.07702743621.42", "ur.0765122623.54", "ur.07627375053.67", "ur.0726615536.45", "ur.072625614.99", "ur.072502766.03", "ur.066617566.80", "ur.0657144310.29", "ur.0647455641.12", "ur.0623024275.39", "ur.016614775072.65", "ur.016537413672.61", "ur.016502357006.97", "ur.016453351545.49", "ur.016444450513.00", "ur.016077077076.94", "ur.016042325224.69", "ur.015416215440.03", "ur.015404021625.15", "ur.015365122232.53", "ur.015275637173.10", "ur.014643304546.52", "ur.014516353037.17", "ur.014436235307.54", "ur.014341562367.11", "ur.014306751412.78", "ur.013603073224.19", "ur.01357722116.76", "ur.01355544706.81", "ur.01343554730.25", "ur.01341665345.10", "ur.01337060205.02", "ur.01335163471.44", "ur.01333564455.68", "ur.01314577014.49", "ur.01313017405.44", "ur.013126150216.26", "ur.013033144331.95", "ur.013024323730.08", "ur.012774637025.41", "ur.01273610367.58", "ur.01271700337.71", "ur.012627700214.51", "ur.01246272657.45", "ur.01243520216.75", "u

0-58 / 58 (5.83s)
===
Records extracted: 58
Starting iteration with limit=1000 skip=0 ...


Current Working Id(s): 
ur.0726615536.45, ur.01243520216.75, ur.01227264605.01, ur.01013220042.13
++++++++There are 4 Working IDs+++++++++++


===
Records extracted: 0
Starting iteration with limit=1000 skip=0 ...
===
Records extracted: 0
Starting iteration with limit=1000 skip=0 ...
===
Records extracted: 0
Starting iteration with limit=1000 skip=0 ...
===
Records extracted: 0


There are 0 number of duplicated patents removed
Final set has 0 number of patents



set()

In [65]:
database_name = "mit-ps+DimensionsCT"
column_names = {
    "abstract":"str", 
    "acronym":"str", 
    "active_years":"int", 
    "altmetric":"float", 
    "associated_grant_ids":"str", 
    "brief_title":"str", 
    "conditions":"str", 
    "date_inserted":"str", 
    "dimensions_url":"str", 
    "end_date":"str", 
    "funder_countries":"str", 
    "funders":"str", 
    "gender":"str", 
    "id":"str", 
    "interventions":"str", 
    "investigators":"str", 
    "linkout":"str", 
    "mesh_terms":"str", 
    "phase":"str", 
    "publication_ids":"str", 
    "publications":"str", 
    "registry":"str", 
    "research_orgs":"str", 
    "researchers":"str",
    "score":"float", 
    "start_date":"str", 
    "study_arms":"str", 
    "study_designs":"str", 
    "study_eligibility_criteria":"str", 
    "study_maximum_age":"str", 
    "study_minimum_age":"str", 
    "study_outcome_measures":"str", 
    "study_participants":"int", 
    "study_type":"str", 
    "title":"str", 
    "category_bra":"str", 
    "category_for":"str", 
    "category_for_2020":"str", 
    "category_hra":"str", 
    "category_hrcs_hc":"str", 
    "category_hrcs_rac":"str", 
    "category_icrp_cso":"str", 
    "category_icrp_ct":"str", 
    "category_rcdc":"str"
}


In [79]:
def execute_command(query):
    try:
        connection = mysql.connector.connect(host='sql.mit.edu',
                                            database=database_name,
                                            user='mit-ps',
                                            passwd='cut18vuk',
                                            charset='utf8')

        # Check if the connection is established
        if connection.is_connected():
            # print("Connection to database established")

            # Create a cursor and execute the insert statement
            cursor = connection.cursor()
            # print('This is current command: ',query)
            cursor.execute(query)
            connection.commit()  # Commit the transaction
            # print("Data inserted successfully")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

    finally:
        # Close the connection
        if connection.is_connected():
            cursor.close()
            connection.close()
            # print("MySQL connection is closed")


def insert_query(table_name, column, value):
    table_name = table_name.replace(' ','_')
    if type(value) == json:
        value = json.dumps(value)
    elif isinstance(value, str):
        value = f"'{value}'"  # Enclose string values in single quotes

    query = f"INSERT INTO {table_name} ({column}) VALUES ({value});"
    return query

In [89]:
def dimensionsTrialSearch(name):
    table_name = name.replace(' ','_')
    execute_command(f"""
    CREATE TABLE {table_name} (
        abstract TEXT,
        acronym VARCHAR(255),
        active_years VARCHAR(255),
        altmetric FLOAT,
        associated_grant_ids TEXT,
        brief_title TEXT,
        conditions TEXT,
        date_inserted DATE,
        dimensions_url VARCHAR(255),
        end_date DATE,
        funder_countries TEXT,
        funders TEXT,
        gender VARCHAR(50),
        id VARCHAR(255),
        interventions TEXT,
        investigators TEXT,
        linkout VARCHAR(255),
        mesh_terms TEXT,
        phase VARCHAR(255),
        publication_ids TEXT,
        publications TEXT,
        registry VARCHAR(255),
        research_orgs TEXT,
        researchers TEXT,
        score FLOAT,
        start_date DATE,
        study_arms TEXT,
        study_designs TEXT,
        study_eligibility_criteria TEXT,
        study_maximum_age VARCHAR(255),
        study_minimum_age VARCHAR(255),
        study_outcome_measures TEXT,
        study_participants INT,
        study_type VARCHAR(255),
        title TEXT,
        category_bra TEXT,
        category_for TEXT,
        category_for_2020 TEXT,
        category_hra TEXT,
        category_hrcs_hc TEXT,
        category_hrcs_rac TEXT,
        category_icrp_cso TEXT,
        category_icrp_ct TEXT,
        category_rcdc TEXT
    );
    """)

    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    resJson = res.json['researchers']

    allIds = [item['id'] for item in resJson]
    allIds_str = ', '.join([f'"{item}"' for item in allIds])
    print(f'Here are all the ids for {name}: ')
    print('[' + allIds_str + ']')

    res2 = dsl.query_iterative(f"""search researchers where id in [{allIds_str}] return researchers[id+obsolete]""")
    res2Json = res2.json['researchers']
    
    allWorkingIDs = [person['id'] for person in res2Json if person['obsolete'] == 0]
    print(f'Current Working Id(s): ')
    print(', '.join(allWorkingIDs))
    print(f'++++++++There are {len(allWorkingIDs)} Working IDs+++++++++++')  


    for id in allWorkingIDs:
        query = dsl.query_iterative(f"""search clinical_trials
                                        where researchers.id = "{id}"
                                        return clinical_trials[basics+categories+extras+studies]
                                        """)
        
        trials = query.json.get('clinical_trials', [])

        for current_trial in trials:
            total_values = []
            for tag in current_trial:
                value = current_trial[tag]
                if not isinstance(value, str):
                    # Convert non-string values to JSON string
                    value = json.dumps(value)
                  
                value = value.replace("'", "\\'")

                # Enclose all values in single quotes
                total_values.append(f"'{value}'")

            columns = ', '.join([f"`{tag}`" for tag in current_trial.keys()])
            values = ', '.join(total_values)
            insert_query = f"INSERT INTO `{table_name}` ({columns}) VALUES ({values});"
            print("SQL Query: ", insert_query)  # Print the query for debugging
            # Execute the query
            execute_command(insert_query)

dimensionsTrialSearch("Michael Agus")


Starting iteration with limit=1000 skip=0 ...


MySQL connection is closed


0-23 / 23 (0.38s)
===
Records extracted: 23
Starting iteration with limit=1000 skip=0 ...


Here are all the ids for Michael Agus: 
["ur.0716114011.47", "ur.0611721335.42", "ur.016265562545.48", "ur.016122063025.72", "ur.015174074015.16", "ur.015063642442.23", "ur.014603530671.80", "ur.014553657160.09", "ur.014505300325.09", "ur.014477610304.89", "ur.014301404030.68", "ur.013350537100.02", "ur.012477564160.41", "ur.011651461740.64", "ur.011641760512.33", "ur.011441741204.33", "ur.011404421435.48", "ur.011206232463.13", "ur.010526602307.50", "ur.010376225637.06", "ur.010340444237.06", "ur.010336561325.81", "ur.01032474040.28"]


0-23 / 23 (4.50s)
===
Records extracted: 23
Starting iteration with limit=1000 skip=0 ...


Current Working Id(s): 
ur.0611721335.42
++++++++There are 1 Working IDs+++++++++++


0-8 / 8 (6.02s)
===
Records extracted: 8


SQL Query:  INSERT INTO `Michael_Agus` (`abstract`, `active_years`, `category_bra`, `category_for`, `category_hra`, `category_hrcs_hc`, `category_rcdc`, `conditions`, `gender`, `id`, `investigators`, `linkout`, `phase`, `registry`, `study_designs`, `study_eligibility_criteria`, `study_maximum_age`, `study_minimum_age`, `study_outcome_measures`, `study_participants`, `study_type`, `title`) VALUES ('The purpose of this study is to learn more about changes in glucose levels in hospitalized infants with intestinal failure receiving parenteral nutrition or PN (nutrients delivered intravenously), as they transition from continuous PN (given 24 hours a day) to cycled PN (given less than 24 hours a day).There is an increased risk of glucose abnormalities with cycled PN, which can be harmful to infant growth and brain health. Continuous glucose monitors (CGM) will be used to measure interstitial glucose levels (in the tissue under the skin), which are similar to blood glucose levels. CGM is a s